# Imports

In [1]:
# pip install duckdb, rdflib, sparqlwrapper
# apt install g++, python3-dev

import requests
import json
import time
from rdflib.namespace import RDF
from rdflib import Graph
from datetime import datetime

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

import duckdb
import os

# SPARQL Queries

## Configuration

In [2]:
from config import *

In [3]:
endpoint_url

'https://gebouwen.brabantcloud.nl/proxy/wdqs/bigdata/namespace/wdq/sparql'

## Helper Functions

In [4]:
def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    sparql.setMethod("POST")
    return sparql.query().convert()


def get_total(endpoint_url, query="""select (count(?s) as ?count) where {?s ?p ?o} """):
    results = get_results(endpoint_url, query)
    total = 0

    for result in results["results"]["bindings"]:
        total = result["count"]['value']

    return int(total)

def get_graph(endpoint_url, query, timeout=10):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST")
    sparql.setTimeout(timeout)
    return sparql.query().convert()

In [5]:
enabled_q_codes = "wd:Q9 wd:Q8 wd:Q7 wd:Q5827 wd:Q5828 wd:Q5829 wd:Q5831 wd:Q5832"
where_query = f"""
 ?building wdt:P1 ?instanceOf ;
  VALUES ?instanceOf {{ {enabled_q_codes} }}.
  ?building wdt:P24 ?arkUri .
  ?building schema:dateModified ?dateModified .
  ?building wdt:P25 wd:Q5557 . # 'Gepubliceerd'
"""

def get_total_k3_entities():
    count_query = """SELECT (count(?building) as ?count) WHERE {{ {} }}""".format(where_query)
    return get_total(endpoint_url, count_query)

# test function
get_total_k3_entities()

1046

## TermenNetwerk

In [6]:
# The query to get the schema.org record for the NDE termennetwerk
schema_construct_query = """
PREFIX bc: <https://data.brabantcloud.nl/wikibase#>
PREFIX wdt: <http://gebouwen.brabantcloud.nl/prop/direct/>
PREFIX wd: <http://gebouwen.brabantcloud.nl/entity/>
PREFIX schema: <https://schema.org/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

CONSTRUCT {
  ?arkUri a schema:LandmarksOrHistoricalBuildings ;
    schema:additionalType ?aatType ;
    schema:name ?buildingName ;
    schema:alternateName ?alternateBuildingName ;
    schema:description ?dateCreatedDateDestroyedDescription ;
    skos:hiddenLabel ?slug ;
    skos:scopeNote ?scopeNote, ?addressScopeNote;
    schema:address ?placeURI.
  
   ?placeURI a schema:PostalAddress ;
      schema:streetAddress ?streetAddressName ;
      schema:addressLocality ?placeName ;
      schema:addressRegion ?municipalityName .
}
WHERE {
  BIND (<<SUBJECT>> AS ?targetURI)
  ?building wdt:P1 ?instanceOf ;
    wdt:P24 ?targetURI; # remove later
        VALUES ?instanceOf { wd:Q9 wd:Q8 wd:Q7 wd:Q5827 wd:Q5828 wd:Q5829 wd:Q5831 wd:Q5832}.
  OPTIONAL { wd:Q9 wdt:P28 ?aatType }
  ?building wdt:P24 ?arkUri .
  ?building rdfs:label ?buildingName .
  OPTIONAL { ?building skos:altLabel ?alternateBuildingName }
OPTIONAL {
    ?building p:P8/psv:P8 ?dateCreatedTimeNode .
    ?dateCreatedTimeNode wikibase:timePrecision ?dateCreatedTimePrecision ;
	    wikibase:timeValue ?dateCreatedTime .
    BIND(YEAR(?dateCreatedTime) AS ?dateCreatedYear)
    BIND(
      COALESCE(
        IF(?dateCreatedTimePrecision = 6, CONCAT(STR(CEIL(?dateCreatedYear / 1000)), "e millennium"), 1/0),
        IF(?dateCreatedTimePrecision = 7, CONCAT(STR(CEIL(?dateCreatedYear / 100)), "e eeuw"), 1/0),
        IF(?dateCreatedTimePrecision = 8, CONCAT("jaren ", STR(?dateCreatedYear)), 1/0),
        IF(?dateCreatedTimePrecision = 9, STR(?dateCreatedYear), 1/0),
        IF(?dateCreatedTimePrecision = 10, CONCAT(STR(MONTH(?dateCreatedTime)), "-", STR(?dateCreatedYear)), 1/0),
        IF(?dateCreatedTimePrecision = 11, CONCAT(STR(DAY(?dateCreatedTime)), "-", STR(MONTH(?dateCreatedTime)), "-", STR(?dateCreatedYear)), STR(?dateCreatedTime))
      )  
    AS ?dateCreated)
   BIND(CONCAT("Gebouwd: ", STR(?dateCreated)) AS ?dateCreatedDescription)
  }
  
  OPTIONAL {
    ?building p:P9/psv:P9 ?dateDestroyedTimeNode .
    ?dateDestroyedTimeNode wikibase:timePrecision ?dateDestroyedTimePrecision ;
	    wikibase:timeValue ?dateDestroyedTime .
    BIND(YEAR(?dateDestroyedTime) AS ?dateDestroyedYear)
    BIND(
      COALESCE(
        IF(?dateDestroyedTimePrecision = 6, CONCAT(STR(CEIL(?dateDestroyedYear / 1000)), "e millennium"), 1/0),
        IF(?dateDestroyedTimePrecision = 7, CONCAT(STR(CEIL(?dateDestroyedYear / 100)), "e eeuw"), 1/0),
        IF(?dateDestroyedTimePrecision = 8, CONCAT("jaren ", STR(?dateDestroyedYear)), 1/0),
        IF(?dateDestroyedTimePrecision = 9, STR(?dateDestroyedYear), 1/0),
        IF(?dateDestroyedTimePrecision = 10, CONCAT(STR(MONTH(?dateDestroyedTime)), "-", STR(?dateDestroyedYear)), 1/0),
        IF(?dateDestroyedTimePrecision = 11, CONCAT(STR(DAY(?dateDestroyedTime)), "-", STR(MONTH(?dateDestroyedTime)), "-", STR(?dateDestroyedYear)), STR(?dateDestroyedTime))
      )  
    AS ?dateDestroyed)
    BIND(CONCAT("Vernietigd: ", STR(?dateDestroyed)) AS ?dateDestroyedDescription)
  }
  BIND(
    COALESCE(
      IF(BOUND(?dateCreatedDescription) && BOUND(?dateDestroyedDescription), CONCAT(?dateCreatedDescription, ". ", ?dateDestroyedDescription), 1/0),
      IF(BOUND(?dateCreatedDescription), ?dateCreatedDescription, 1/0),
      IF(BOUND(?dateDestroyedDescription), ?dateDestroyedDescription, 1/0)
    )
  AS ?dateCreatedDateDestroyedDescription)
  OPTIONAL { ?building wdt:P3 ?streetAddressName }
  OPTIONAL {
    ?building wdt:P4 ?place .
    ?place rdfs:label ?placeName .
    OPTIONAL {
      ?place wdt:P5 ?municipality .
      ?municipality rdfs:label ?municipalityName
    }
  }
  BIND (URI(CONCAT(STR(?arkUri), "/", "address")) as ?placeURI) .
  BIND(
    COALESCE(
      IF(BOUND(?streetAddressName) && BOUND(?placeName) && BOUND(?municipalityName), CONCAT("Adres: ", ?streetAddressName, ", ", ?placeName, ", ", ?municipalityName), ?noAddress),
      IF(BOUND(?streetAddressName) && BOUND(?placeName), CONCAT("Adres: ", ?streetAddressName, ", ", ?placeName), ?noAddress),
      IF(BOUND(?streetAddressName), CONCAT("Adres: ", ?streetAddressName), ?noAddress),
      IF(BOUND(?placeName) && BOUND(?municipalityName), CONCAT("Plaats: ", ?placeName, ", ", ?municipalityName), ?noAddress),
      IF(BOUND(?placeName), CONCAT("Plaats: ", ?placeName), ?noAddress)
    ) AS ?addressScopeNote
  )
  OPTIONAL {
    ?building p:P33/psv:P33 ?dateInUseTimeNode .
    ?dateInUseTimeNode wikibase:timePrecision ?dateInUseTimePrecision ;
	    wikibase:timeValue ?dateInUseTime .
    BIND(YEAR(?dateInUseTime) AS ?dateInUseYear)
    BIND(
      COALESCE(
        IF(?dateInUseTimePrecision = 6, CONCAT(STR(CEIL(?dateInUseYear / 1000)), "e millennium"), 1/0),
        IF(?dateInUseTimePrecision = 7, CONCAT(STR(CEIL(?dateInUseYear / 100)), "e eeuw"), 1/0),
        IF(?dateInUseTimePrecision = 8, CONCAT("jaren ", STR(?dateInUseYear)), 1/0),
        IF(?dateInUseTimePrecision = 9, STR(?dateInUseYear), 1/0),
        IF(?dateInUseTimePrecision = 10, CONCAT(STR(MONTH(?dateInUseTime)), "-", STR(?dateInUseYear)), 1/0),
        IF(?dateInUseTimePrecision = 11, CONCAT(STR(DAY(?dateInUseTime)), "-", STR(MONTH(?dateInUseTime)), "-", STR(?dateInUseYear)), STR(?dateInUseTime))
      )  
    AS ?dateInUse)
  }  
  
  OPTIONAL {
    ?building p:P34/psv:P34 ?dateNoLongerInUseTimeNode .
    ?dateNoLongerInUseTimeNode wikibase:timePrecision ?dateNoLongerInUseTimePrecision ;
	    wikibase:timeValue ?dateNoLongerInUseTime .
    BIND(YEAR(?dateNoLongerInUseTime) AS ?dateNoLongerInUseYear)
    BIND(
      COALESCE(
        IF(?dateNoLongerInUseTimePrecision = 6, CONCAT(STR(CEIL(?dateNoLongerInUseYear / 1000)), "e millennium"), 1/0),
        IF(?dateNoLongerInUseTimePrecision = 7, CONCAT(STR(CEIL(?dateNoLongerInUseYear / 100)), "e eeuw"), 1/0),
        IF(?dateNoLongerInUseTimePrecision = 8, CONCAT("jaren ", STR(?dateNoLongerInUseYear)), 1/0),
        IF(?dateNoLongerInUseTimePrecision = 9, STR(?dateNoLongerInUseYear), 1/0),
        IF(?dateNoLongerInUseTimePrecision = 10, CONCAT(STR(MONTH(?dateNoLongerInUseTime)), "-", STR(?dateNoLongerInUseYear)), 1/0),
        IF(?dateNoLongerInUseTimePrecision = 11, CONCAT(STR(DAY(?dateNoLongerInUseTime)), "-", STR(MONTH(?dateNoLongerInUseTime)), "-", STR(?dateNoLongerInUseYear)), STR(?dateNoLongerInUseTime))
      )  
    AS ?dateNoLongerInUse)
            }  
    # plaats, adres - voorkeursnaam gebouw (datum ingebruikname – datum uitgebruikname)
  BIND(
     COALESCE(
        IF(BOUND(?placeName) && BOUND(?streetAddressName) && BOUND(?buildingName), CONCAT(?placeName, ", ", ?streetAddressName, " - ", ?buildingName), 1/0),
        IF(BOUND(?placeName) && BOUND(?buildingName), CONCAT(?placeName, ", ", "onbekend", " - ", ?buildingName), 1/0),
        IF(BOUND(?streetAddressName) && BOUND(?buildingName), CONCAT("onbekend", ", ", ?streetAddressName, " - ", ?buildingName), 1/0),
        IF(BOUND(?buildingName), CONCAT("plaats onbekend", ", ", "adres onbekend", " - ", ?buildingName), 1/0)
     )
    AS ?slugPrefix)
  BIND(
    COALESCE(
      # full case
      IF(BOUND(?dateInUse) && BOUND(?dateNoLongerInUse), CONCAT("(", ?dateInUse, " - ", ?dateNoLongerInUse, ")"), 1/0),
      IF(BOUND(?dateInUse), CONCAT("(", ?dateInUse, " - heden)"), 1/0),
      IF(BOUND(?dateNoLongerInUse), CONCAT("(onbekend - ", ?dateNoLongerInUse, ")"), 1/0)
    )
  AS ?slugSuffix)
  BIND(
    COALESCE(
      IF(BOUND(?slugPrefix) && BOUND(?slugSuffix), CONCAT(?slugPrefix, " ", ?slugSuffix), 1/0),
      IF(BOUND(?slugPrefix), ?slugPrefix, 1/0)
    )
  AS ?slug)
  BIND(
      COALESCE(
        IF(?instanceOf = <http://gebouwen.brabantcloud.nl/entity/Q7>, "Type: Kerkgebouw", 1/0),
        IF(?instanceOf = <http://gebouwen.brabantcloud.nl/entity/Q8>, "Type: Kapel", 1/0),
        IF(?instanceOf = <http://gebouwen.brabantcloud.nl/entity/Q9>, "Type: Kloostergebouw", 1/0),
        IF(?instanceOf = <http://gebouwen.brabantcloud.nl/entity/Q5827>, "Type: Kerktoren", 1/0),
        IF(?instanceOf = <http://gebouwen.brabantcloud.nl/entity/Q5828>, "Type: Gebouwencomplex", 1/0),
        IF(?instanceOf = <http://gebouwen.brabantcloud.nl/entity/Q5829>, "Type: Pastorie", 1/0),
        IF(?instanceOf = <http://gebouwen.brabantcloud.nl/entity/Q5831>, "Type: Standbeeld", 1/0),
        IF(?instanceOf = <http://gebouwen.brabantcloud.nl/entity/Q5832>, "Type: Heilig Hartbeeld", 1/0)
      )  
    AS ?scopeNote )
  ?building wdt:P25 wd:Q5557 # 'Gepubliceerd'
}
LIMIT 1000
"""

In [7]:
def get_termennetwerk_record(uri):
    query = schema_construct_query.replace("<<SUBJECT>>", f"<{uri}>")
    return get_graph(endpoint_url, query)

# test for termennetwork query
get_termennetwerk_record("https://data.brabantcloud.nl/id/ark:15052/554c3cad-9033-4817-a8c0-571f8daf06ab").serialize()

'@prefix schema: <https://schema.org/> .\n@prefix skos: <http://www.w3.org/2004/02/skos/core#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n<https://data.brabantcloud.nl/id/ark:15052/554c3cad-9033-4817-a8c0-571f8daf06ab> a schema:LandmarksOrHistoricalBuildings ;\n    skos:hiddenLabel "Boxmeer, Spoorstraat 65a - Kerk van de Evangelische Gemeente (2e millennium - 2e millennium)"^^xsd:string ;\n    skos:scopeNote "Adres: Spoorstraat 65a, Boxmeer, Gemeente Land van Cuijk"^^xsd:string,\n        "Type: Kerkgebouw"^^xsd:string ;\n    schema:additionalType <http://vocab.getty.edu/aat/300000641> ;\n    schema:address <https://data.brabantcloud.nl/id/ark:15052/554c3cad-9033-4817-a8c0-571f8daf06ab/address> ;\n    schema:description "Gebouwd: 2e millennium"^^xsd:string ;\n    schema:name "Kerk van de Evangelische Gemeente"@nl .\n\n<https://data.brabantcloud.nl/id/ark:15052/554c3cad-9033-4817-a8c0-571f8daf06ab/address> a schema:PostalAddress ;\n    schema:addressLocality "Boxmeer"@nl ;\

## MappingXML

In [8]:
construct_query_1 = """
PREFIX bc: <https://data.brabantcloud.nl/wikibase#>
PREFIX p: <http://gebouwen.brabantcloud.nl/prop/>
PREFIX psv: <http://gebouwen.brabantcloud.nl/prop/statement/value/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <https://schema.org/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX wdt: <http://gebouwen.brabantcloud.nl/prop/direct/>
PREFIX wd: <http://gebouwen.brabantcloud.nl/entity/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX custom: <https://data.brabantcloud.nl/ns/custom#>

CONSTRUCT {
  ?arkUri a schema:LandmarksOrHistoricalBuildings ;
    custom:wikibaseURI ?building;
    # names
    schema:name ?nameOfBuilding ;
    schema:alternateName ?alternateBuildingName ;
    # types
    schema:additionalType ?aatType ;
    custom:typeOfBuilding ?typeOfBuilding ;  # P1


    # usage status
    custom:partOfURI ?partOfURI;    # P29
    custom:hasPartURI ?hasPartURI;  # P30
    custom:replacesPreviousURI ?replacesPreviousURI; # P32
    custom:isReplacedByURI ?isReplacedByURI; # P31
    
    
    custom:monumentStatus ?monumentStatus;  # P17
    custom:monumentLinkRCE ?monumentRCELink ; # P17 > P22
    
    
    # external entities
    custom:dedicationURI ?dedicationURI;    # P11
    custom:architectURI ?architectURI ;     # P10
    custom:religionURI ?religionURI ;       # P15
    custom:imageURI ?imageURI;              # P6
    custom:religiousOrder ?religiousOrderURI;  # P16
    custom:placeURI ?placeURI.
  
  # image entity
  ?imageURI a custom:WebResource;
     skos:prefLabel ?imageLabel;        # P6
     custom:rightsLabel ?imageRights;   # P13
     custom:rightsURI ?imageRightsLink; # P14
     custom:creator ?imageCreator;      # P12
     custom:imageLink ?imageLink;       # P7
     custom:wikiBaseURI ?image.
  
  # architect entity
  ?architectURI a custom:Agent;
      skos:prefLabel ?architectLabel;        # P10 rdfs:label
      custom:wikidata ?architectWikiDataURI;
      skos:altLabel ?architectAltLabel.      # P10 skos:altLabel
  
  # religion entity
  ?religionURI a skos:Concept;
      skos:prefLabel ?religionLabel;
      custom:wikidata ?religionWikiDataURI;
      skos:altLabel ?religionAltLabel.

  # dedication entity
  ?dedicationURI a skos:Concept;
      skos:prefLabel ?dedicationLabel;
      custom:wikidata ?dedicationWikiDataURI;
      skos:altLabel ?dedicationAltLabel.
  
  # religious order entity
  ?religiousOrderURI a skos:Concept;
      skos:prefLabel ?orderLabel;
      custom:wikidata ?orderWikiDataURI;
      skos:altLabel ?orderAltLabel.
  
  # place entity   
  ?placeURI a schema:PostalAddress ;
     schema:streetAddress ?streetAddressName ;  # P3
     schema:addressLocality ?nameOfPlace ;      # P4
     schema:addressRegion ?nameOfMunicipality . # P5
}

WHERE {
  # Bind to the K3 type
  BIND(<<URI>> AS ?building)
 
  # make sure to filter the right types 
  ?building wdt:P1 ?instanceOf ;
    VALUES ?instanceOf {wd:Q7 wd:Q8 wd:Q9 wd:Q5827 wd:Q5828 wd:Q5829 wd:Q5831 wd:Q5832 } . # Kerkgebouw, Kapel, Kerkgebouw
 
  # [START primary metadata] # 
  ?building wdt:P24 ?arkUri .
  # names
  ?building rdfs:label ?nameOfBuilding .
  OPTIONAL { ?building skos:altLabel ?alternateBuildingName }
  # types
  ?instanceOf rdfs:label ?typeOfBuilding .
  OPTIONAL { ?instanceOf wdt:P28 ?aatType }
  # monument status 
  OPTIONAL {
    ?building wdt:P17 ?monumentStatusLink.
    ?monumentStatusLink rdfs:label ?monumentStatus .
  }
  # RCE monument link
  OPTIONAL {
    ?building <http://gebouwen.brabantcloud.nl/prop/P17> ?statement1 .
    ?statement1 prov:wasDerivedFrom ?statement2 .
    ?statement2 <http://gebouwen.brabantcloud.nl/prop/reference/P22> ?monumentRCELink .
  }
  # [END primary metadata] # 
  
  # [START usage status] #
  OPTIONAL { 
      ?building wdt:P29 ?partOf.
       ?partOf wdt:P24 ?partOfURI .
  }
  OPTIONAL { 
      ?building wdt:P30 ?hasPart.
       ?hasPart wdt:P24 ?hasPartURI.
  }
  OPTIONAL { 
      ?building wdt:P31 ?replacedBy.
       ?replacedBy wdt:P24 ?isReplacedByURI .
  }
  OPTIONAL { 
      ?building wdt:P32 ?replacesPrevious.
       ?replacesPrevious wdt:P24 ?replacesPreviousURI.
  }
  # [END usage status] #
  
  # [START attribution] #
  OPTIONAL {
    ?building wdt:P11 ?dedication.
    ?dedication wdt:P24 ?dedicationURI;
                wdt:P25 wd:Q5557.
    OPTIONAL { ?dedication rdfs:label ?dedicationLabel}
    OPTIONAL { ?dedication  wdt:P21 ?dedicationWikiDataURI}
    OPTIONAL { ?dedication  skos:altLabel ?dedicationAltLabel}
  }
  # [END attribution] #
  
  # [START architect] #
  OPTIONAL {
    ?building wdt:P10 ?architect.
    ?architect wdt:P24 ?architectURI;
               wdt:P25 wd:Q5557.
     OPTIONAL { ?architect     rdfs:label ?architectLabel}
     OPTIONAL { ?architect      wdt:P21 ?architectWikiDataURI}
     OPTIONAL { ?architect      skos:altLabel ?architectAltLabel}
               
  }
  # [END architect] #
  
  # [START thumbnails] #
  OPTIONAL {
    ?building wdt:P6 ?image.
    ?image wdt:P24 ?imageURI;
           wdt:P25 wd:Q5557.
    OPTIONAL { ?image rdfs:label ?imageLabel }
    OPTIONAL { ?image wdt:P13 ?imageRights }
    OPTIONAL { ?image wdt:P14 ?imageRightsLink }
    OPTIONAL { ?image wdt:P12 ?imageCreator }
    OPTIONAL { ?image wdt:P7 ?imageLink}
  }
  # [END thumbnails] #
 
  # [START religion ] #
    OPTIONAL {
    ?building wdt:P15 ?religion.
    ?religion wdt:P24 ?religionURI;
              wdt:P25 wd:Q5557.
    OPTIONAL { ?religion rdfs:label ?religionLabel}
    OPTIONAL { ?religion wdt:P21 ?religionWikiDataURI}
    OPTIONAL { ?religion skos:altLabel ?religionAltLabel}
               
  }
  # [END religion ] #
  
  # [START religious order] #
  OPTIONAL {
    ?building wdt:P16 ?religiousOrder.
    ?religiousOrder wdt:P24 ?religiousOrderURI;
              wdt:P25 wd:Q5557.
    OPTIONAL { ?religiousOrder rdfs:label ?orderLabel}
    OPTIONAL { ?religiousOrder wdt:P21 ?orderWikiDataURI}
    OPTIONAL { ?religiousOrder skos:altLabel ?orderAltLabel}
               
  }
  # [END religious order] #
  
  # [START places] #
  OPTIONAL { ?building wdt:P3 ?streetAddressName }
  OPTIONAL {
    ?building wdt:P4 ?place .
    ?place wdt:P24 ?placeURI .
    ?place rdfs:label ?nameOfPlace .
    OPTIONAL {
      ?place wdt:P5 ?municipality .
      ?municipality rdfs:label ?nameOfMunicipality .
    }
  }
  # [END places] #

}
ORDER BY ?typeOfBuilding
LIMIT 500
    """

In [9]:
construct_query_2 = """
PREFIX bc: <https://data.brabantcloud.nl/wikibase#>
PREFIX p: <http://gebouwen.brabantcloud.nl/prop/>
PREFIX psv: <http://gebouwen.brabantcloud.nl/prop/statement/value/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <https://schema.org/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX wdt: <http://gebouwen.brabantcloud.nl/prop/direct/>
PREFIX wd: <http://gebouwen.brabantcloud.nl/entity/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX custom: <https://data.brabantcloud.nl/ns/custom#>

CONSTRUCT {
  ?arkUri a schema:LandmarksOrHistoricalBuildings ; 
    # dates
    custom:dateCreated ?dateCreated;  # P8
    custom:dateInUse ?dateInUse;      # P33
    custom:dateNoLongerInUse ?dateNoLongerInUse; # P34
    custom:dateDestroyed ?dateDestroyed;  # P9
            
    # dates raw 
    custom:dateCreatedRaw ?dateCreatedTime;
    custom:dateInUseRaw ?dateInUseTime;
    custom:dateNoLongerInUseRaw ?dateNoLongerInUseTime;
    custom:dateDestroyedRaw ?dateDestroyedTime;
            
    # misc
    custom:latLong ?latLong;   # P2
    custom:rightsURI ?rightsURI;
    
    # links URIs
    custom:mipURI ?mipURI;            # P18
    custom:reliWiki ?reliWikiURI;     # P27 
    custom:dbpedia ?dbpediaURI;       # P20 
    custom:wikidata ?wikidataURI;     # P21
    custom:regioWiki ?regioWikiURI.      # P19
}

WHERE {
  # Bind to the K3 type
  BIND(<<URI>> AS ?building)
 
  # make sure to filter the right types 
  ?building wdt:P1 ?instanceOf ;
    VALUES ?instanceOf {wd:Q7 wd:Q8 wd:Q9 wd:Q5827 wd:Q5828 wd:Q5829 wd:Q5831 wd:Q5832 } . # Kerkgebouw, Kapel, Kerkgebouw
 
  # [START primary metadata] # 
  ?building wdt:P24 ?arkUri .
  # coordinates
  OPTIONAL { ?building wdt:P2 ?latLong }
  # rights
  OPTIONAL { ?building wdt:P6 ?rightsURI }
  # [END primary metadata] # 

  # [START linked URIs] #
  OPTIONAL { ?building wdt:P18 ?mipURI }
  OPTIONAL { ?building wdt:P27 ?reliWikiURI }
  OPTIONAL { ?building wdt:P20 ?dbpediaURI }
  OPTIONAL { ?building wdt:P21 ?wikidataURI }
  OPTIONAL { ?building wdt:P19 ?regioWikiURI }
  # [END linked URIs] #

  # [START dates] # 
  OPTIONAL {
    ?building p:P8/psv:P8 ?dateCreatedTimeNode .
    ?dateCreatedTimeNode wikibase:timePrecision ?dateCreatedTimePrecision ;
	    wikibase:timeValue ?dateCreatedTime .
    BIND(YEAR(?dateCreatedTime) AS ?dateCreatedYear)
    BIND(
      COALESCE(
        IF(?dateCreatedTimePrecision = 6, CONCAT(STR(CEIL(?dateCreatedYear / 1000)), "e millennium"), 1/0),
        IF(?dateCreatedTimePrecision = 7, CONCAT(STR(CEIL(?dateCreatedYear / 100)), "e eeuw"), 1/0),
        IF(?dateCreatedTimePrecision = 8, CONCAT("jaren ", STR(?dateCreatedYear)), 1/0),
        IF(?dateCreatedTimePrecision = 9, STR(?dateCreatedYear), 1/0),
        IF(?dateCreatedTimePrecision = 10, CONCAT(STR(MONTH(?dateCreatedTime)), "-", STR(?dateCreatedYear)), 1/0),
        IF(?dateCreatedTimePrecision = 11, CONCAT(STR(DAY(?dateCreatedTime)), "-", STR(MONTH(?dateCreatedTime)), "-", STR(?dateCreatedYear)), STR(?dateCreatedTime))
      )  
    AS ?dateCreated)
  }
  
  OPTIONAL {
    ?building p:P9/psv:P9 ?dateDestroyedTimeNode .
    ?dateDestroyedTimeNode wikibase:timePrecision ?dateDestroyedTimePrecision ;
	    wikibase:timeValue ?dateDestroyedTime .
    BIND(YEAR(?dateDestroyedTime) AS ?dateDestroyedYear)
    BIND(
      COALESCE(
        IF(?dateDestroyedTimePrecision = 6, CONCAT(STR(CEIL(?dateDestroyedYear / 1000)), "e millennium"), 1/0),
        IF(?dateDestroyedTimePrecision = 7, CONCAT(STR(CEIL(?dateDestroyedYear / 100)), "e eeuw"), 1/0),
        IF(?dateDestroyedTimePrecision = 8, CONCAT("jaren ", STR(?dateDestroyedYear)), 1/0),
        IF(?dateDestroyedTimePrecision = 9, STR(?dateDestroyedYear), 1/0),
        IF(?dateDestroyedTimePrecision = 10, CONCAT(STR(MONTH(?dateDestroyedTime)), "-", STR(?dateDestroyedYear)), 1/0),
        IF(?dateDestroyedTimePrecision = 11, CONCAT(STR(DAY(?dateDestroyedTime)), "-", STR(MONTH(?dateDestroyedTime)), "-", STR(?dateDestroyedYear)), STR(?dateDestroyedTime))
      )  
    AS ?dateDestroyed)
  }
  
  OPTIONAL {
    ?building p:P33/psv:P33 ?dateInUseTimeNode .
    ?dateInUseTimeNode wikibase:timePrecision ?dateInUseTimePrecision ;
	    wikibase:timeValue ?dateInUseTime .
    BIND(YEAR(?dateInUseTime) AS ?dateInUseYear)
    BIND(
      COALESCE(
        IF(?dateInUseTimePrecision = 6, CONCAT(STR(CEIL(?dateInUseYear / 1000)), "e millennium"), 1/0),
        IF(?dateInUseTimePrecision = 7, CONCAT(STR(CEIL(?dateInUseYear / 100)), "e eeuw"), 1/0),
        IF(?dateInUseTimePrecision = 8, CONCAT("jaren ", STR(?dateInUseYear)), 1/0),
        IF(?dateInUseTimePrecision = 9, STR(?dateInUseYear), 1/0),
        IF(?dateInUseTimePrecision = 10, CONCAT(STR(MONTH(?dateInUseTime)), "-", STR(?dateInUseYear)), 1/0),
        IF(?dateInUseTimePrecision = 11, CONCAT(STR(DAY(?dateInUseTime)), "-", STR(MONTH(?dateInUseTime)), "-", STR(?dateInUseYear)), STR(?dateInUseTime))
      )  
    AS ?dateInUse)
  }  
  
  OPTIONAL {
    ?building p:P34/psv:P34 ?dateNoLongerInUseTimeNode .
    ?dateNoLongerInUseTimeNode wikibase:timePrecision ?dateNoLongerInUseTimePrecision ;
	    wikibase:timeValue ?dateNoLongerInUseTime .
    BIND(YEAR(?dateNoLongerInUseTime) AS ?dateNoLongerInUseYear)
    BIND(
      COALESCE(
        IF(?dateNoLongerInUseTimePrecision = 6, CONCAT(STR(CEIL(?dateNoLongerInUseYear / 1000)), "e millennium"), 1/0),
        IF(?dateNoLongerInUseTimePrecision = 7, CONCAT(STR(CEIL(?dateNoLongerInUseYear / 100)), "e eeuw"), 1/0),
        IF(?dateNoLongerInUseTimePrecision = 8, CONCAT("jaren ", STR(?dateNoLongerInUseYear)), 1/0),
        IF(?dateNoLongerInUseTimePrecision = 9, STR(?dateNoLongerInUseYear), 1/0),
        IF(?dateNoLongerInUseTimePrecision = 10, CONCAT(STR(MONTH(?dateNoLongerInUseTime)), "-", STR(?dateNoLongerInUseYear)), 1/0),
        IF(?dateNoLongerInUseTimePrecision = 11, CONCAT(STR(DAY(?dateNoLongerInUseTime)), "-", STR(MONTH(?dateNoLongerInUseTime)), "-", STR(?dateNoLongerInUseYear)), STR(?dateNoLongerInUseTime))
      )  
    AS ?dateNoLongerInUse)
  }  
  # [END dates] #   
}
ORDER BY ?typeOfBuilding
LIMIT 1000
    """

In [10]:
def get_record(uri):
    query_1 = construct_query_1.replace("<<URI>>", f"<{uri}>")
    g = get_graph(endpoint_url, query_1, timeout=10)
    query_2 = construct_query_2.replace("<<URI>>", f"<{uri}>")
    g2 = get_graph(endpoint_url, query_2, timeout=10)
    g+=g2
    return g

# test function
get_record("http://gebouwen.brabantcloud.nl/entity/Q2473").serialize()

'@prefix custom: <https://data.brabantcloud.nl/ns/custom#> .\n@prefix geo: <http://www.opengis.net/ont/geosparql#> .\n@prefix schema: <https://schema.org/> .\n@prefix skos: <http://www.w3.org/2004/02/skos/core#> .\n@prefix wd: <http://gebouwen.brabantcloud.nl/entity/> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n<https://data.brabantcloud.nl/id/ark:15052/6bb12195-0db1-4dc9-9f95-3ab47734ddd6> a schema:LandmarksOrHistoricalBuildings ;\n    custom:architectURI <https://data.brabantcloud.nl/id/ark:15052/f691e0d0-6849-4fcf-8aa9-639eac9a4e07> ;\n    custom:dateCreated "1964"^^xsd:string ;\n    custom:dateCreatedRaw "1964-01-01T00:00:00+00:00"^^xsd:dateTime ;\n    custom:dateInUse "1964"^^xsd:string ;\n    custom:dateInUseRaw "1964-01-01T00:00:00+00:00"^^xsd:dateTime ;\n    custom:dateNoLongerInUse "5-4-2010"^^xsd:string ;\n    custom:dateNoLongerInUseRaw "2010-04-05T00:00:00+00:00"^^xsd:dateTime ;\n    custom:dedicationURI <https://data.brabantcloud.nl/id/ark:15052/a6881884-78b7-

# Sync (duckDB)

In [11]:
def setup_db_tables(con, cursor, reset_all=False):
    table_name = 'k3_records'
    if reset_all:
        try:
            cursor.execute(f"PRAGMA table_info({table_name})")
            if len(cursor.fetchall()) > 0:
                cursor.execute(f'DROP TABLE {table_name}')
        except Exception as e:
            print(e)

    # Create the table if it doesn't exist
    cursor.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (
                   wikibaseURI VARCHAR unique, 
                   arkURI VARCHAR unique,
                   lastModified TIMESTAMP,
                   lastSeen TIMESTAMP,
                   deleted boolean DEFAULT FALSE, 
                   ndeData blob, 
                   narthexData blob,
                   ndeInError varchar,
                   narthexInError varchar,
                   )''')
    con.commit()

def get_lastmodified_record_date(cursor):
    # Query to check data
    result = cursor.execute("SELECT MAX(lastModified) AS lastModified FROM k3_records;").fetchall()
    lastModifiedRecord = datetime(1971, 1, 1, 0, 0)
    if result[0][0]:
        lastModifiedRecord = result[0][0]

    print("lastModified date: {}".format(lastModifiedRecord))
    return lastModifiedRecord



In [37]:
def update_lastmodified_from_wikibase(con, cursor, debug=False, override_lastModified=None):
    harvest_query = """SELECT distinct ?building ?arkUri ?dateModified WHERE {{ {} }} ORDER BY ?building """.format(where_query)
    total =  get_total_k3_entities()
    start = 0
    page = 2500

    previous_last_date = override_lastModified
    if not previous_last_date:
       previous_last_date = get_lastmodified_record_date(cursor)

    # stats
    new_record = updated = unmodified = total_seen = 0
    start_time = datetime.now()
    time.sleep(2)

    while start < total:
        pager = "limit {} offset {}".format(page, start)
        if debug:
            print(harvest_query+pager)
        results = get_results(endpoint_url, harvest_query+pager)
        bindings = results['results']['bindings']
        for b in bindings:
            ark_uri = b['arkUri']['value']
            wikibase_uri = b['building']['value']
            last_modified = b['dateModified']['value']
            last_modified_dt = datetime.strptime(last_modified, "%Y-%m-%dT%H:%M:%SZ")
            row = (wikibase_uri, ark_uri, last_modified, datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            total_seen+=1
            knownRecord = cursor.execute("SELECT * FROM k3_records WHERE arkURI = ?", (ark_uri,)).fetchone()
            if not knownRecord:
                new_record+=1
                con.execute("INSERT INTO k3_records (wikibaseURI, arkURI, lastModified, lastSeen) VALUES (?, ?, ?, ?)", row)
            elif knownRecord[2] != last_modified_dt or knownRecord[2] > previous_last_date:
                updated+=1
                con.execute("UPDATE k3_records SET lastSeen = CURRENT_TIMESTAMP, deleted = false, lastModified = ? WHERE arkURI = ?", (last_modified_dt, ark_uri,))

            else:
                # print(knownRecord)
                unmodified+=1
                con.execute("UPDATE k3_records SET lastSeen = CURRENT_TIMESTAMP, deleted = false WHERE arkURI = ?", (ark_uri,))

        start += page
        if start % 5000 == 0:
            print("harvested {} from {}".format(start, total))

    con.execute(f"UPDATE k3_records SET deleted = true where lastSeen <= '{start_time}';")

    res = con.execute("select count(*) from k3_records where deleted = true;").fetchone()
    deleted = res[0]
    print(f"total: {total_seen}; new records: {new_record}; modified {updated}; unmodified: {unmodified}; deleted {deleted}")

    return previous_last_date

In [13]:
def count_modified_records(con, lastModifiedDateTime):
    dt_str = lastModifiedDateTime.strftime('%Y-%m-%d %H:%M:%S')
    # Query timestamps in DuckDB that are later than the given datetime object
    modified_records = con.execute(f"SELECT count(*) FROM k3_records WHERE lastModified > '{dt_str}'").fetchone()
    all_records_count = con.execute("select count(*) from k3_records").fetchone()
    return modified_records[0], all_records_count[0]

# Sync with K3 (Wikibase)

In [14]:
total_buildings = get_total_k3_entities()
print(f"total buildings in wikibase: {total_buildings}")

total buildings in wikibase: 1046


## Get modified records

In [24]:
def get_modified_arks(con, lastModified):
    arks, _ = get_modified_ids(con, lastModified)
    return arks

def get_modified_subjects(con, lastModified):
    _, wikibase_uris = get_modified_ids(con, lastModified)
    return wikibase_uris


def get_modified_ids(con, lastModified):
    if not lastModified:
        lastModified = datetime(1970, 1, 1, 0, 0, 0)
    res = con.execute(f"select wikibaseURI, arkURI from k3_records where lastModified > '{lastModified}' and deleted = false").fetchall()
    arks = []
    wikibase_uris = []
    for row in res:
        arks.append(row[1])
        wikibase_uris.append(row[0])

    return arks, wikibase_uris

## helper functions

In [16]:
def generate_mapping_xml(con, ignore=[]):
    seen = 0
    with open(mapping_xml_output_fname, 'wb') as output:
        output.write("<records>\n".encode("utf-8"))

        all_records_count = con.execute("select count(*) from k3_records where deleted = false and narthexData <> ''").fetchone()[0]
        for rec in con.execute("select wikibaseURI, narthexData from k3_records where deleted = false and narthexData <> ''").fetchall():
            subj = rec[0]
            data = rec[1]

            if not data:
                continue

            parts = subj.split("/")
            id = parts[len(parts)-1]
            if id in ignore:
                print("ignoring: {}".format(id))
                continue

            try:
                #print(subj)
                output.write('<record id="{}">\n'.format(id).encode("utf-8"))
                output.write(data.replace(b'<?xml version="1.0" encoding="utf-8"?>\n', b'').replace(b'https://data.brabantcloud.nl/id/ark:', b'https://n2t.net/ark:/'))
                output.write("</record>\n".encode("utf-8"))
            except Exception as err:
                errors[id] = err
                print(count, id)
                # print(err)

            seen += 1
            if seen % 25 == 0:
                print("seen {} of {}".format(seen, all_records_count))
                # break

        output.write("</records>".encode("utf-8"))

In [17]:
def update_k3_termennetwerk_endpoint(endpoint_url=k3_termennetwerk_update_endpoint_url, fname=k3_termennetwerk_output_fname):
    # store the data using put and the graph store protocol
    headers = {
        'Content-Type': 'text/plain',
        'charset': "utf-8"
    }

    with open(fname, 'rb') as ntriples_file:
        response = requests.put(endpoint_url, headers=headers, data=ntriples_file, timeout=10, allow_redirects=True)

    if response.status_code == 200:
        print('Data uploaded successfully.')
    else:
        print(f'An error occurred: {response.content} [{response.status_code}]')


In [26]:
# %%time 
def update_k3_termennetwerk_triples(con, lastModified):
    seen = 0

    arks = get_modified_arks(con, lastModified)
    for ark in arks:
        g1 = get_termennetwerk_record(ark)
        data = g1.serialize(encoding='utf-8', format='turtle')
        con.execute("UPDATE k3_records SET ndeData = ? WHERE arkURI = ?", (data, ark,))
        seen += 1 
        if seen % 25 == 0:
            print("seen {} of {}".format(seen, len(arks)))
            #break

    print(f"update {seen} records for termennetwerk")

def store_k3_termennetwerk_triples(con, fname=k3_termennetwerk_output_fname):
    schema_graph = Graph()
    seen = 0

    all_records_count = con.execute("select count(*) from k3_records where deleted = false").fetchone()[0]
    for rec in con.execute("select ndeData from k3_records where deleted = false").fetchall():
        data = rec[0]
        seen += 1
        if not data:
            continue

        schema_graph.parse(data=data.replace(b'https://data.brabantcloud.nl/id/ark:', b'https://n2t.net/ark:/'), format='turtle')
        if seen % 25 == 0:
            print("seen {} of {}".format(seen, all_records_count))
            #break

    # the file that needs to be uploaded to k3-2 sparql endpoint
    with open(fname, 'wb') as output:
        schema_graph.serialize(output, 'nt', encoding="utf-8")

In [19]:
k3_termennetwerk_output_fname

'./k3_schema.nt'

In [20]:
# %%time 

# probably remove later
def update_mapping_xml_records(con, lastModified):
    q_uris = get_modified_subjects(con, lastModified)

    ignore = []

    q_uris = sorted(q_uris)
    errors = {}

    seen = 0
    total = len(q_uris)
    for count, subj in enumerate(q_uris):
        store_mapping_xml(con, subj, errors=errors, count=count)
        seen+=1

        if seen % 10 == 0:
            print(f"processing {seen} records of {total}")

    print(f"stored {seen} mappingxml records")

def store_mapping_xml(con, subj, count=0, ignore_q_uris=[], errors={}):
    parts = subj.split("/")
    id = parts[len(parts)-1]
    if id in ignore_q_uris:
        print("ignoring: {}".format(id))
        return

    try:
        #print(subj)
        res = con.execute("select narthexData from k3_records where wikibaseURI = ?", (subj,)).fetchone()
        storedData = res[0]
        #if storedData:
            #continue
        g1 = get_record(subj)
        data = g1.serialize(format='pretty-xml', encoding='utf-8')
        if data != storedData:
            con.execute("UPDATE k3_records SET narthexData = ?, narthexInError = '' WHERE wikibaseURI = ?", (data, subj,))
    except Exception as err:
        errors[id] = err
        con.execute("UPDATE k3_records SET narthexInError = ?, narthexData = '' WHERE wikibaseURI = ?", ("{}".format(err), subj,))
        print(count, id)
        print(err)




## Run full sync

In [21]:
def full_sync(reset_all=False, override_lastModified=None):
    # open the database
    con = duckdb.connect(database=os.path.join(work_directory, 'k3.duckdb'), read_only=False)
    cursor = con.cursor()

    try:
        # setup or update db tables
        setup_db_tables(con, cursor, reset_all)

        # sync db with update status from wikibase
        print("sync with wikibase")
        last_modified = update_lastmodified_from_wikibase(con, cursor, override_lastModified=override_lastModified)

        # process records for termennetwerk
        print("update modified with sparql for nde-termennetwerk from wikibase")
        update_k3_termennetwerk_triples(con, last_modified)

        # process records for mapping xml
        print("update modified with sparql for nde-termennetwerk from wikibase")
        update_mapping_xml_records(con, last_modified)

        # store k3 termennetwerk triples
        print("create termennetwerk turtle file")
        store_k3_termennetwerk_triples(con)
        
        # update k3_termennetwerk sparql endpoint
        print("update termennetwerk via sparql")
        update_k3_termennetwerk_endpoint(endpoint_url=k3_termennetwerk_update_endpoint_url)
        
        # update mapping records
        print("generated mapping_full.xml file for narthex download")
        generate_mapping_xml(con)
    except Exception as e:
        print("stopping with an error")
        print(e)
    finally:
        # close the database
        cursor.close()
        con.close()

    print(f"finished running update: {datetime.now()}")

In [22]:
override_lastModified = datetime(2023,9,4,7,42,10)

In [41]:
# %%time
full_sync(False, override_lastModified=None)

sync with wikibase
lastModified date: 2023-09-25 12:51:44
total: 1030; new records: 0; modified 0; unmodified: 1030; deleted 1
update modified with sparql for nde-termennetwerk from wikibase
update 0 records for termennetwerk
update modified with sparql for nde-termennetwerk from wikibase
stored 0 mappingxml records
create termennetwerk turtle file
seen 25 of 1030
seen 50 of 1030
seen 75 of 1030
seen 100 of 1030
seen 125 of 1030
seen 150 of 1030
seen 175 of 1030
seen 200 of 1030
seen 225 of 1030
seen 250 of 1030
seen 275 of 1030
seen 300 of 1030
seen 325 of 1030
seen 350 of 1030
seen 375 of 1030
seen 400 of 1030
seen 425 of 1030
seen 450 of 1030
seen 475 of 1030
seen 500 of 1030
seen 525 of 1030
seen 550 of 1030
seen 575 of 1030
seen 600 of 1030
seen 625 of 1030
seen 650 of 1030
seen 675 of 1030
seen 700 of 1030
seen 725 of 1030
seen 750 of 1030
seen 775 of 1030
seen 800 of 1030
seen 825 of 1030
seen 850 of 1030
seen 875 of 1030
seen 900 of 1030
seen 925 of 1030
seen 950 of 1030
seen 9